# ResNet model initial training

In [3]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import os
import numpy as np
from glob import glob
import json

from keras.applications.resnet50 import ResNet50

In [4]:
def extract_Resnet50(tensor):
    from keras.applications.resnet50 import ResNet50, preprocess_input
    return ResNet50_model_base.predict(preprocess_input(tensor))  

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(bottleneck_feature))

def ResNet50_feature_vec(img_path):
    # returns feature vector for image located at img_path
    return extract_Resnet50(path_to_tensor(img_path))



In [5]:
train_targets = np.load(open('./images/train_targets.npy','rb'))
valid_targets = np.load(open('./images/valid_targets.npy','rb'))
test_targets = np.load(open('./images/test_targets.npy','rb'))

original_train = np.load(open('./images/original_train.npy','rb'))
original_validation = np.load(open('./images/original_validation.npy','rb'))
original_test = np.load(open('./images/original_test.npy', 'rb'))

bottleneck_ResNet50_train = np.load(open('./bottleneck_features/bottleneck_ResNet50_train.npy','rb'))
bottleneck_ResNet50_validation = np.load(open('./bottleneck_features/bottleneck_ResNet50_validation.npy','rb'))
bottleneck_ResNet50_test = np.load(open('./bottleneck_features/bottleneck_ResNet50_test.npy','rb'))

In [6]:
from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

def assessAccuracy(model, features, targets):
    predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in features]
    test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(targets, axis=1))/len(predictions)
    print('Test accuracy: %.4f%%' % test_accuracy)
    
    from sklearn.metrics import f1_score
    print('f1_score:')
    print(f1_score([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import confusion_matrix
    print('confusion matrix')
    print(confusion_matrix([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import average_precision_score
    print('Precision')
    print(average_precision_score([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import recall_score
    print('Recall')
    print(recall_score([np.argmax(r) for r in targets], predictions))

## Initial training of ResNet model using bottleneck features

In [7]:

# define Xception model
resnet_model_base = ResNet50(weights='imagenet', include_top=False)

In [8]:
# Resnet 1
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


resnet1_model = Sequential()
resnet1_model.add(Flatten(input_shape=bottleneck_ResNet50_train.shape[1:]))
resnet1_model.add(Dropout(0.7))
resnet1_model.add(Dense(2, activation='softmax'))

resnet1_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 4,098.0
Trainable params: 4,098.0
Non-trainable params: 0.0
_________________________________________________________________


In [27]:
### TODO: Compile the model.
resnet1_model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=[f1])

In [21]:
from keras.callbacks import ModelCheckpoint  

### TODO: Train the model.
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.resnet1.hdf5', 
                               verbose=1, save_best_only=True)

#weight the importance of the malignant/benign samples so that the model does not become bias to benign which makes up 80% of samples
# 1,2,3 input_class_weight= {0:1., 1:10.}
input_class_weight= {0:1., 1:4.}

resnet1_model.fit(bottleneck_ResNet50_train, train_targets, 
          validation_data=(bottleneck_ResNet50_validation, valid_targets),
          epochs=1000, batch_size=50, class_weight=input_class_weight, callbacks=[checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/1000
2000/2000 [==============================] - 0s - loss: 2.0568 - f1: 0.5595 - val_loss: 0.5575 - val_f1: 0.7333
Epoch 2/1000
2000/2000 [==============================] - 0s - loss: 1.5715 - f1: 0.5930 - val_loss: 0.5773 - val_f1: 0.6667
Epoch 3/1000
2000/2000 [==============================] - 0s - loss: 1.4375 - f1: 0.6260 - val_loss: 0.5082 - val_f1: 0.8000
Epoch 4/1000
2000/2000 [==============================] - 0s - loss: 1.3455 - f1: 0.6385 - val_loss: 0.6382 - val_f1: 0.6667
Epoch 5/1000
2000/2000 [==============================] - 0s - loss: 1.2090 - f1: 0.6375 - val_loss: 0.5209 - val_f1: 0.7467
Epoch 6/1000
2000/2000 [==============================] - 0s - loss: 1.1483 - f1: 0.6830 - val_loss: 0.5099 - val_f1: 0.7400
Epoch 7/1000
2000/2000 [==============================] - 0s - loss: 1.0531 - f1: 0.6860 - val_loss: 0.4834 - val_f1: 0.7867
Epoch 8/1000
2000/2000 [==============================] - 0s - loss: 1.1533 - 

2000/2000 [==============================] - 0s - loss: 1.0044 - f1: 0.7190 - val_loss: 0.8021 - val_f1: 0.5933
Epoch 66/1000
2000/2000 [==============================] - 0s - loss: 0.9521 - f1: 0.7215 - val_loss: 0.6080 - val_f1: 0.6467
Epoch 67/1000
2000/2000 [==============================] - 0s - loss: 0.9886 - f1: 0.7160 - val_loss: 0.6116 - val_f1: 0.6467
Epoch 68/1000
2000/2000 [==============================] - 0s - loss: 0.9611 - f1: 0.7220 - val_loss: 0.5513 - val_f1: 0.6800
Epoch 69/1000
2000/2000 [==============================] - 0s - loss: 0.9759 - f1: 0.7190 - val_loss: 0.5036 - val_f1: 0.8200
Epoch 70/1000
2000/2000 [==============================] - 0s - loss: 0.9155 - f1: 0.7355 - val_loss: 0.4949 - val_f1: 0.7733
Epoch 71/1000
2000/2000 [==============================] - 0s - loss: 0.8934 - f1: 0.7340 - val_loss: 0.4657 - val_f1: 0.7800
Epoch 72/1000
2000/2000 [==============================] - 0s - loss: 0.9934 - f1: 0.7290 - val_loss: 0.4918 - val_f1: 0.7667
Epoch 

2000/2000 [==============================] - 0s - loss: 0.9310 - f1: 0.7385 - val_loss: 0.6207 - val_f1: 0.6733
Epoch 132/1000
2000/2000 [==============================] - 0s - loss: 0.9924 - f1: 0.7240 - val_loss: 0.5986 - val_f1: 0.6867
Epoch 133/1000
2000/2000 [==============================] - 0s - loss: 0.9345 - f1: 0.7275 - val_loss: 0.5025 - val_f1: 0.7533
Epoch 134/1000
2000/2000 [==============================] - 0s - loss: 0.9636 - f1: 0.7275 - val_loss: 0.5095 - val_f1: 0.7600
Epoch 135/1000
2000/2000 [==============================] - 0s - loss: 1.0359 - f1: 0.7055 - val_loss: 0.4968 - val_f1: 0.7733
Epoch 136/1000
2000/2000 [==============================] - 0s - loss: 1.0051 - f1: 0.7150 - val_loss: 0.5322 - val_f1: 0.8067
Epoch 137/1000
2000/2000 [==============================] - 0s - loss: 1.0192 - f1: 0.7075 - val_loss: 0.6120 - val_f1: 0.6867
Epoch 138/1000
2000/2000 [==============================] - 0s - loss: 0.9800 - f1: 0.7230 - val_loss: 0.5238 - val_f1: 0.7600

2000/2000 [==============================] - 0s - loss: 0.9661 - f1: 0.7195 - val_loss: 0.5674 - val_f1: 0.7000
Epoch 198/1000
2000/2000 [==============================] - 0s - loss: 0.9700 - f1: 0.7235 - val_loss: 0.5496 - val_f1: 0.7400
Epoch 199/1000
2000/2000 [==============================] - 0s - loss: 0.9993 - f1: 0.7135 - val_loss: 0.5271 - val_f1: 0.7800
Epoch 200/1000
2000/2000 [==============================] - 0s - loss: 0.9307 - f1: 0.7170 - val_loss: 0.5520 - val_f1: 0.7133
Epoch 201/1000
2000/2000 [==============================] - 0s - loss: 0.9456 - f1: 0.7125 - val_loss: 0.5698 - val_f1: 0.7000
Epoch 202/1000
2000/2000 [==============================] - 0s - loss: 0.9817 - f1: 0.7180 - val_loss: 0.6608 - val_f1: 0.6400
Epoch 203/1000
2000/2000 [==============================] - 0s - loss: 0.9993 - f1: 0.7210 - val_loss: 0.5252 - val_f1: 0.7333
Epoch 204/1000
2000/2000 [==============================] - 0s - loss: 1.0487 - f1: 0.7105 - val_loss: 0.5175 - val_f1: 0.7800

2000/2000 [==============================] - 0s - loss: 0.9911 - f1: 0.7265 - val_loss: 0.6125 - val_f1: 0.6733
Epoch 264/1000
2000/2000 [==============================] - 0s - loss: 0.9773 - f1: 0.7250 - val_loss: 0.5574 - val_f1: 0.7267
Epoch 265/1000
2000/2000 [==============================] - 0s - loss: 0.9623 - f1: 0.7210 - val_loss: 0.5559 - val_f1: 0.7200
Epoch 266/1000
2000/2000 [==============================] - 0s - loss: 0.9345 - f1: 0.7345 - val_loss: 0.5393 - val_f1: 0.7467
Epoch 267/1000
2000/2000 [==============================] - 0s - loss: 0.9289 - f1: 0.7360 - val_loss: 0.5820 - val_f1: 0.7000
Epoch 268/1000
2000/2000 [==============================] - 0s - loss: 1.0023 - f1: 0.7165 - val_loss: 0.5604 - val_f1: 0.7333
Epoch 269/1000
2000/2000 [==============================] - 0s - loss: 0.9343 - f1: 0.7325 - val_loss: 0.5656 - val_f1: 0.8133
Epoch 270/1000
2000/2000 [==============================] - 0s - loss: 1.0109 - f1: 0.7280 - val_loss: 0.6288 - val_f1: 0.6867

...

2000/2000 [==============================] - 0s - loss: 0.9967 - f1: 0.7240 - val_loss: 0.5517 - val_f1: 0.7200
Epoch 726/1000
2000/2000 [==============================] - 0s - loss: 0.9969 - f1: 0.7235 - val_loss: 0.5216 - val_f1: 0.7600
Epoch 727/1000
2000/2000 [==============================] - 0s - loss: 0.9684 - f1: 0.7235 - val_loss: 0.6402 - val_f1: 0.6400
Epoch 728/1000
2000/2000 [==============================] - 0s - loss: 1.0257 - f1: 0.7185 - val_loss: 0.5252 - val_f1: 0.8067
Epoch 729/1000
2000/2000 [==============================] - 0s - loss: 0.9798 - f1: 0.7235 - val_loss: 0.5719 - val_f1: 0.7000
Epoch 730/1000
2000/2000 [==============================] - 0s - loss: 0.9905 - f1: 0.7140 - val_loss: 0.5471 - val_f1: 0.7200
Epoch 731/1000
2000/2000 [==============================] - 0s - loss: 0.9698 - f1: 0.7255 - val_loss: 0.7248 - val_f1: 0.6267
Epoch 732/1000
2000/2000 [==============================] - 0s - loss: 0.9458 - f1: 0.7185 - val_loss: 0.5242 - val_f1: 0.7600

2000/2000 [==============================] - 0s - loss: 0.9835 - f1: 0.7280 - val_loss: 0.5542 - val_f1: 0.7400
Epoch 792/1000
2000/2000 [==============================] - 0s - loss: 0.9969 - f1: 0.7030 - val_loss: 0.6041 - val_f1: 0.6733
Epoch 793/1000
2000/2000 [==============================] - 0s - loss: 1.0089 - f1: 0.7030 - val_loss: 0.4961 - val_f1: 0.7800
Epoch 794/1000
2000/2000 [==============================] - 0s - loss: 1.0160 - f1: 0.7100 - val_loss: 0.4928 - val_f1: 0.7733
Epoch 795/1000
2000/2000 [==============================] - 0s - loss: 1.0093 - f1: 0.7255 - val_loss: 0.5023 - val_f1: 0.7600
Epoch 796/1000
2000/2000 [==============================] - 0s - loss: 1.0188 - f1: 0.7165 - val_loss: 0.5484 - val_f1: 0.7067
Epoch 797/1000
2000/2000 [==============================] - 0s - loss: 0.9276 - f1: 0.7285 - val_loss: 0.5737 - val_f1: 0.7067
Epoch 798/1000
2000/2000 [==============================] - 0s - loss: 1.0600 - f1: 0.7070 - val_loss: 0.7881 - val_f1: 0.5933

2000/2000 [==============================] - 0s - loss: 1.0054 - f1: 0.7320 - val_loss: 0.6671 - val_f1: 0.6800
Epoch 858/1000
2000/2000 [==============================] - 0s - loss: 1.0012 - f1: 0.7275 - val_loss: 0.5594 - val_f1: 0.7667
Epoch 859/1000
2000/2000 [==============================] - 0s - loss: 1.0968 - f1: 0.7030 - val_loss: 0.5989 - val_f1: 0.6933
Epoch 860/1000
2000/2000 [==============================] - 0s - loss: 1.0130 - f1: 0.7130 - val_loss: 0.5459 - val_f1: 0.7400
Epoch 861/1000
2000/2000 [==============================] - 0s - loss: 0.9939 - f1: 0.7065 - val_loss: 0.5420 - val_f1: 0.7933
Epoch 862/1000
2000/2000 [==============================] - 0s - loss: 1.0484 - f1: 0.7150 - val_loss: 0.5287 - val_f1: 0.8200
Epoch 863/1000
2000/2000 [==============================] - 0s - loss: 0.9512 - f1: 0.7560 - val_loss: 0.7998 - val_f1: 0.5800
Epoch 864/1000
2000/2000 [==============================] - 0s - loss: 0.9802 - f1: 0.7330 - val_loss: 0.5578 - val_f1: 0.7467

2000/2000 [==============================] - 0s - loss: 1.0192 - f1: 0.7160 - val_loss: 0.5313 - val_f1: 0.7400
Epoch 924/1000
2000/2000 [==============================] - 0s - loss: 1.0862 - f1: 0.7130 - val_loss: 0.5370 - val_f1: 0.7467
Epoch 925/1000
2000/2000 [==============================] - 0s - loss: 1.0267 - f1: 0.7165 - val_loss: 0.5712 - val_f1: 0.7200
Epoch 926/1000
2000/2000 [==============================] - 0s - loss: 1.0276 - f1: 0.7185 - val_loss: 0.5842 - val_f1: 0.7133
Epoch 927/1000
2000/2000 [==============================] - 0s - loss: 0.9710 - f1: 0.7090 - val_loss: 0.5689 - val_f1: 0.7467
Epoch 928/1000
2000/2000 [==============================] - 0s - loss: 1.0223 - f1: 0.7165 - val_loss: 0.5438 - val_f1: 0.7533
Epoch 929/1000
2000/2000 [==============================] - 0s - loss: 1.0591 - f1: 0.7085 - val_loss: 0.6886 - val_f1: 0.6467
Epoch 930/1000
2000/2000 [==============================] - 0s - loss: 1.0198 - f1: 0.7050 - val_loss: 0.6379 - val_f1: 0.6733

2000/2000 [==============================] - 0s - loss: 0.9787 - f1: 0.7260 - val_loss: 0.5496 - val_f1: 0.7133
Epoch 990/1000
2000/2000 [==============================] - 0s - loss: 1.0177 - f1: 0.7170 - val_loss: 0.5504 - val_f1: 0.8067
Epoch 991/1000
2000/2000 [==============================] - 0s - loss: 1.0613 - f1: 0.7035 - val_loss: 0.5678 - val_f1: 0.7133
Epoch 992/1000
2000/2000 [==============================] - 0s - loss: 0.9727 - f1: 0.7190 - val_loss: 0.5255 - val_f1: 0.7733
Epoch 993/1000
2000/2000 [==============================] - 0s - loss: 1.0008 - f1: 0.7275 - val_loss: 0.5760 - val_f1: 0.7000
Epoch 994/1000
2000/2000 [==============================] - 0s - loss: 0.9788 - f1: 0.7355 - val_loss: 0.6858 - val_f1: 0.6400
Epoch 995/1000
2000/2000 [==============================] - 0s - loss: 1.0619 - f1: 0.6985 - val_loss: 0.5245 - val_f1: 0.7800
Epoch 996/1000
2000/2000 [==============================] - 0s - loss: 1.0405 - f1: 0.7205 - val_loss: 0.5989 - val_f1: 0.7000

In [9]:
resnet1_model.load_weights('saved_models/weights.best.resnet1.hdf5')


In [10]:
assessAccuracy(resnet1_model, bottleneck_ResNet50_test, test_targets)

Test accuracy: 80.0000%
f1_score:
0.464285714286
confusion matrix
[[428  53]
 [ 67  52]]
Precision
0.52193977591
Recall
0.436974789916


### Fine tune resnet model

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input
processed_train = preprocess_input(original_train)
processed_test = preprocess_input(original_test)
processed_validation = preprocess_input(original_validation)

In [55]:

resnet_model_base.summary()
print(len(resnet_model_base.layers))

# unfreeze weights of the 5th layer
layers_to_freeze = 34 
# unfreeze the weights of the 4th and 5th layers.
#layers_to_freeze = 96
#try freezing half the forth layer + 5th.
#layers_to_freeze = 65



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D) (None, 230, 230, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256                                          
___________________________________________________________________________________________

In [56]:
from keras.models import Model
from keras import optimizers
from keras.layers import Input

# create the base pre-trained model
input_tensor = Input(shape=original_train[0].shape)
resnet_model_base = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)
resnet1_model.load_weights('saved_models/weights.best.resnet1.hdf5')

last_layer_to_use_pretrained = len(resnet_model_base.layers) - layers_to_freeze

# set the first x layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in resnet_model_base.layers[:last_layer_to_use_pretrained]:
    layer.trainable = False
for layer in resnet_model_base.layers[last_layer_to_use_pretrained:]:
    layer.trainable = True
    
fine_tuned_resnet_model = Model(inputs=resnet_model_base.input, output= resnet1_model(resnet_model_base.output))

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
fine_tuned_resnet_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=[f1])

//anaconda/envs/capstone-project/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  '` call to the Keras 2 API: ' + signature)


In [52]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.resnet.final.hdf5', 
                               verbose=1, save_best_only=True)

#weight the importance of the malignant/benign samples so that the model does not become bias to benign which makes up 80% of samples
input_class_weight= {0:1., 1:4.}

fine_tuned_resnet_model.fit(original_train, train_targets, 
          validation_data=(original_validation, valid_targets),
          epochs=200, batch_size=20, class_weight=input_class_weight, callbacks=[checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/200
2000/2000 [==============================] - 3567s - loss: 2.9705 - f1: 0.6100 - val_loss: 0.5085 - val_f1: 0.7933
Epoch 2/200
2000/2000 [==============================] - 635s - loss: 1.4647 - f1: 0.6575 - val_loss: 5.4496 - val_f1: nan
Epoch 3/200
2000/2000 [==============================] - 632s - loss: 1.2016 - f1: 0.7140 - val_loss: 12.7173 - val_f1: nan
Epoch 4/200
 440/2000 [=====>........................] - ETA: 464s - loss: 0.9365 - f1: 0.7636

KeyboardInterrupt: 

In [53]:
fine_tuned_resnet_model.load_weights('saved_models/weights.best.resnet.final.hdf5')

In [57]:
assessAccuracy(fine_tuned_resnet_model, original_test, test_targets)

Test accuracy: 80.1667%
f1_score:
0.0
confusion matrix
[[481   0]
 [119   0]]


//anaconda/envs/capstone-project/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



### Generate predictions and store for training/validation
to be used for training our random forrest classifier, and test to be used to validate against. Use the initial model as this is not overfit, like the fine tuned model.

In [58]:
train_pred = resnet1_model.predict(bottleneck_ResNet50_train)
val_pred = resnet1_model.predict(bottleneck_ResNet50_validation)
test_pred = resnet1_model.predict(bottleneck_ResNet50_test)

In [59]:
np.save(open('./cnn_output/resnet_train.npy', 'wb'), train_pred)
np.save(open('./cnn_output/resnet_validation.npy', 'wb'), val_pred)
np.save(open('./cnn_output/resnet_test.npy', 'wb'), test_pred)

[[ 0.41516539  0.58483464]
 [ 0.93360239  0.06639765]
 [ 0.19024831  0.80975169]
 ..., 
 [ 0.99888748  0.00111248]
 [ 0.99826342  0.00173662]
 [ 0.99874425  0.00125573]]
